# 문제 04 유형(DataSet_04.csv 이용)

구분자 : comma(“,”), 6,718 Rows, 4 Columns, UTF-8 인코딩

 한국인의 식생활 변화가 건강에 미치는 영향을 분석하기에 앞서 육류
 소비량에 대한 분석을 하려고 한다. 확보한 데이터는 세계 각국의 1인당
 육류 소비량 데이터로 아래와 같은 내용을 담고 있다.

 컬 럼 / 정 의 / Type
 LOCATION / 국가명 / String
 SUBJECT / 육류 종류 (BEEF / PIG / POULTRY / SHEEP) / String
 TIME / 연도 (1990 ~ 2026) / Integer
 Value / 1인당 육류 소비량 (KG) / Double

In [1]:
import pandas as pd
import numpy as np

data04 = pd.read_csv('C:/Users/joey0/CodeStudy/proDS/Dataset/Dataset_04.csv')

## 1.한국인의 1인당 육류 소비량이 해가 갈수록 증가하는 것으로 보여 상관분석을 통하여 확인하려고 한다. 
 - 데이터 파일로부터 한국 데이터만 추출한다. 한국은 KOR로 표기되어 있다.
 - 년도별 육류 소비량 합계를 구하여 TIME과 Value간의 상관분석을 수행하고 상관계수를 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지만 기술하시오. 

In [2]:
data04.columns

Index(['LOCATION', 'SUBJECT', 'TIME', 'Value'], dtype='object')

In [4]:
q1 = data04[data04.LOCATION == 'KOR']

q1_tab = q1.groupby('TIME')['Value'].sum()

q1_tab2 = q1_tab.reset_index()

In [5]:
q1_tab2.corr()

,TIME,Value
TIME,1.000000,0.960124
Value,0.960124,1.000000


In [6]:
q1_tab2.corr()['TIME']

TIME     1.000000
Value    0.960124
Name: TIME, dtype: float64

In [7]:
q1_tab2.corr()['TIME']['Value']

#답 : 0.96

0.9601244896033333

## 2. 한국 인근 국가 가운데 식생의 유사성이 상대적으로 높은 일본(JPN)과 비교하여, 연도별 소비량에 평균 차이가 있는지 분석하고자 한다.
 - 두 국가의 육류별 소비량을 연도기준으로 비교하는 대응표본 t 검정을 수행하시오.
 - 두 국가 간의 연도별 소비량 차이가 없는 것으로 판단할 수 있는 육류 종류를 모두 적으시오. (알파벳 순서)

In [11]:
from scipy.stats import ttest_rel

q2 = data04[data04.LOCATION.isin(['KOR','JPN'])]

q2.SUBJECT.unique()
#['BEEF', 'PIG', 'POULTRY', 'SHEEP']
sub_list = q2.SUBJECT.unique()

#temp = q2[q2.SUBJECT == 'BEEF']
#q2_tab = pd.pivot_table(temp, index='TIME', columns='LOCATION', values='Value').dropna()

#q2_out = ttest_rel(q2_tab['KOR'], q2_tab['JPN'])

q2_out2 = []

In [14]:
for i in sub_list:
    temp = q2[q2.SUBJECT== i]
    q2_tab = pd.pivot_table(temp, index='TIME', columns='LOCATION', values='Value').dropna()
    
    q2_out = ttest_rel(q2_tab['KOR'], q2_tab['JPN'])
    pvalue = q2_out.pvalue
    q2_out2.append([i, pvalue])

q2_out2 = pd.DataFrame(q2_out2, columns=['sub', 'pvalue'])

q2_out2[q2_out2.pvalue >= 0.05]['sub']
#차이가 있는 경우의 답 : POULTRY

2    POULTRY
6    POULTRY
Name: sub, dtype: object

In [15]:
#[참고]
q2_out2[q2_out2.pvalue < 0.05]['sub']
#차이가 없는 경우의 답 : BEEF, PIG, SHEEP

0     BEEF
1      PIG
3    SHEEP
4     BEEF
5      PIG
7    SHEEP
Name: sub, dtype: object

## 3.(한국만 포함한 데이터에서) Time을 독립변수로, Value를 종속변수로 하여 육류 종류(SUBJECT) 별로 회귀분석을 수행하였을 때, 가장 높은 결정계수를 가진 모델의 학습오차 중 MAPE를 반올림하여 소수점 둘째 자리까지 기술하시오.
 (MAPE : Mean Absolute Percentage Error, 평균 절대 백분율 오차)
 (MAPE = Σ ( | y - y ̂ | / y ) * 100/n ))

In [16]:
from sklearn.linear_model import LinearRegression

q3 = q1.copy() #한국만 포함한 데이터

sub_list = q3.SUBJECT.unique()

q3_out = []

In [17]:
for i in sub_list:
    temp = q3[q3.SUBJECT == i]
    lm = LinearRegression().fit(temp[['TIME']], temp['Value'])
    r2 = lm.score(temp[['TIME']], temp['Value'])
    pred = lm.predict(temp[['TIME']])
    # (MAPE = Σ ( | y - y ̂ | / y ) * 100/n ))
    mape = (abs(temp['Value'] - pred)/temp['Value']).sum() * 100 / len(temp)
    
    q3_out.append([i, r2, mape])   

In [21]:
q3_out = pd.DataFrame(q3_out, columns=['sub','r2','mape'])

q3_out.loc[q3_out.r2.idxmax(),:]

sub      POULTRY
r2      0.951498
mape    5.783358
Name: 2, dtype: object

In [22]:
q3_out.loc[q3_out.r2.idxmax(),'mape']

#정답 : 5.78

5.783357902874552

In [23]:
#[참고]
for i in sub_list:
    temp = q3[q3.SUBJECT == i]
    globals()['lm_'+i] = LinearRegression().fit(temp[['TIME']], temp['Value'])

temp = q3[q3.SUBJECT == 'BEEF']
lm_BEEF.predict(temp[['TIME']])
#eval('lm_'+i).predict(temp[['TIME']])


array([ 5.72341431,  5.87801722,  6.03262013,  6.18722305,  6.34182596,
        6.49642887,  6.65103178,  6.8056347 ,  6.96023761,  7.11484052,
        7.26944343,  7.42404635,  7.57864926,  7.73325217,  7.88785509,
        8.042458  ,  8.19706091,  8.35166382,  8.50626674,  8.66086965,
        8.81547256,  8.97007547,  9.12467839,  9.2792813 ,  9.43388421,
        9.58848712,  9.74309004,  9.89769295, 10.05229586, 10.20689877,
       10.36150169, 10.5161046 , 10.67070751, 10.82531042, 10.97991334,
       11.13451625])